# Inference pipeline

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export 
from xview2comp.nb_03 import *

### Pre-disaster and post-disaster image file paths

In [3]:
SOURCE = Path('data/test/images')
SOURCE.ls()[:3]

[PosixPath('data/test/images/test_pre_00831.png'),
 PosixPath('data/test/images/test_post_00171.png'),
 PosixPath('data/test/images/test_pre_00064.png')]

In [4]:
preimgs = pre_img_fpaths(get_image_files(SOURCE))

In [5]:
posimgs = [pre2post_fpath(o) for o in preimgs]

In [6]:
len(preimgs), len(posimgs)

(933, 933)

In [7]:
preimgs[:3]

[PosixPath('data/test/images/test_pre_00831.png'),
 PosixPath('data/test/images/test_pre_00064.png'),
 PosixPath('data/test/images/test_pre_00091.png')]

In [8]:
posimgs[:3]

[PosixPath('data/test/images/test_post_00831.png'),
 PosixPath('data/test/images/test_post_00064.png'),
 PosixPath('data/test/images/test_post_00091.png')]

### Building detection

In [10]:
#seglearn = load_learner('./', file='building_detector.pkl', test=SegItemListCustom(preimgs), tfm_y=False)
seglearn = load_learner('./', file='sz1024_bs2_resnet34_8epochcycle_stage5.pkl')

In [11]:
#y_mask, pred_mask, raw_pred_mask = seglearn.get_preds(ds_type=DatasetType.Test)

In [48]:
idx = 117

#idx = 67 #all undamaged  
#idx = 34 #no features

In [49]:
img = open_image(preimgs[idx])
print(preimgs[idx])

data/test/images/test_pre_00290.png


In [50]:
%%time
y_mask, pred_mask, raw_pred_mask = seglearn.predict(img)

CPU times: user 632 ms, sys: 40 ms, total: 672 ms
Wall time: 336 ms


In [51]:
_, ax = plt.subplots(figsize=(6, 6))
img.show(ax)
y_mask.show(ax, alpha=.5, cmap=CMAP, vmax=4)

### Building mask -> polygons

In [52]:
_, polys = bmask2polys(y_mask.data[0].numpy())

In [53]:
type(polys), len(polys)

(numpy.ndarray, 12)

### Generate unique ids for polygons

In [54]:
#export
def generate_uid_polys(polys): return np.array([str(uuid4()) for _ in polys])

In [55]:
uids = generate_uid_polys(polys)

### Get polygon images

In [56]:
path = Path('tmp_polyimgs')

In [57]:
shutil.rmtree(path, ignore_errors=True)
os.makedirs(path, exist_ok=True)

In [58]:
img = open_image(posimgs[idx])

In [59]:
for uid, p in zip(uids, polys):
    polyimg = crop_by_polygon(img, p)
    polyimg.save(path/f'{uid}.png')

Refactor into function.

In [60]:
#export
def polyimgs2disk(img, polys, path=None):
    if path is None: path = Path('tmp_polyimgs')
    shutil.rmtree(path, ignore_errors=True); os.makedirs(path, exist_ok=True)
    if len(polys) == 0: return path, []
    uids = generate_uid_polys(polys)
    for uid, p in zip(uids, polys): crop_by_polygon(img, p).save(path/f'{uid}.png')
    return path, uids

In [61]:
path, uids = polyimgs2disk(img, polys, path)

In [62]:
len(path.ls()), len(uids)

(12, 12)

### Damage classification

In [63]:
%%time
il = ImageList([path/f'{uid}.png' for uid in uids])
damglearn = load_learner('./', file='best_model_damg_nsample42000_sz299_bs32_resnet152.pkl', test=il)

CPU times: user 212 ms, sys: 140 ms, total: 352 ms
Wall time: 251 ms


In [64]:
len(il.items)

12

In [65]:
%%time
pred_damg, y_damg = damglearn.get_preds(ds_type=DatasetType.Test)

CPU times: user 116 ms, sys: 332 ms, total: 448 ms
Wall time: 591 ms


In [66]:
pred_classes = [damglearn.data.classes[i] for i in pred_damg.argmax(dim=1)]

In [67]:
pred_classes[3], pred_damg[3], y_damg[3], y_damg.sum()

('major', tensor([-3.4438,  1.8134,  5.3363, -0.1258]), tensor(0), tensor(0))

In [68]:
len(pred_classes), len(uids), len(polys), img

(12, 12, 12, Image (3, 1024, 1024))

Refactor into function.

In [71]:
#export
def assess_damage_polyimgs(polyimgs, path='./', file='stage2_last_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth.pkl'):
    if len(polyimgs.items) == 0: return [] # Need `.items` because `len(ImageList)` is 1 for empty list.
    damglearn = load_learner(path, file, test=polyimgs).to_fp32()
    pred_damg, _ = damglearn.get_preds(ds_type=DatasetType.Test)
    return [damglearn.data.classes[o] for o in pred_damg.argmax(dim=1)]

In [72]:
polyimgs = ImageList([path/f'{uid}.png' for uid in uids])
damgs = assess_damage_polyimgs(polyimgs)

In [73]:
damgs[3], len(damgs)

('major', 12)

### Damage mask

In [74]:
CATS

['nun', 'minor', 'major', 'destroyed']

In [75]:
#export
DAMG_OTOI = {'nun':1, 'minor':2, 'major':3, 'destroyed':4}

In [76]:
damgmask = np.zeros(img.shape[1:], dtype=np.uint8)

In [77]:
for damg, poly in zip(damgs, polys):
    cv2.fillPoly(damgmask, poly[None,...], (DAMG_OTOI[damg]))
    
damgmask = ImageSegment(tensor(damgmask[None,...]))

Refactor into function.

In [78]:
#export
def damgpolys2damgmask(polys, damgs, sz=1024):
    assert len(polys) == len(damgs)
    if isinstance(sz, int): sz = (sz, sz)
    mask = np.zeros(sz, dtype=np.uint8)
    if len(polys) > 0:
        for damg, poly in zip(damgs, polys): 
            cv2.fillPoly(mask, poly[None,...], (DAMG_OTOI[damg]))
    return ImageSegment(tensor(mask[None,...]))

In [79]:
damgmask = damgpolys2damgmask(polys, damgs, img.shape[1:])

In [80]:
damgmask.data.unique()

tensor([0, 3])

In [81]:
_, ax = plt.subplots(figsize=(10, 10))
img.show(ax)
damgmask.show(ax, alpha=.5, cmap=CMAP, vmax=4)

### Pipeline

In [82]:
#export
def infer_sample(path_preimg, path_posimg, seglearn, path_damglearn):
    img = open_image(path_preimg)
    y_mask, _, _ = seglearn.predict(img)
    _, polys = bmask2polys(y_mask.data[0].numpy())
    img = open_image(path_posimg)
    path, uids = polyimgs2disk(img, polys, path=Path('tmp_polyimgs'))
    polyimgs = ImageList([path/f'{uid}.png' for uid in uids])
    damgs = assess_damage_polyimgs(polyimgs, file=path_damglearn)
    damgmask = damgpolys2damgmask(polys, damgs, sz=img.shape[1:])
    return damgmask

def get_imgid(imgname): return imgname.stem.split('_')[-1]

In [104]:
path_results = Path('./inference_results_stage2_best_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth')

In [105]:
os.makedirs(path_results, exist_ok=True)

In [106]:
idx = 332
damgmask = infer_sample(preimgs[idx], posimgs[idx], seglearn, 'stage2_best_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth.pkl')

In [107]:
_, ax = plt.subplots(figsize=(6, 6))
open_image(posimgs[idx]).show(ax)
damgmask.show(ax, alpha=.8, cmap=CMAP, vmax=len(CMAP.colors)-1)
damgmask.data.unique()

tensor([0, 1, 3, 4])

Takes about 40 minutes to generate the inference .pngs for the entire test set.

In [135]:
path_damglearn = Path('best_model_damg_nsample94313_sz299_bs32_resnet152.pkl')
path_results = Path(f'./inference_results_{path_damglearn.stem}')
print(path_damglearn)
print(str(path_results))

best_model_damg_nsample94313_sz299_bs32_resnet152.pkl
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152


In [124]:
path_damglearn = Path('best_model_damg_nsample94313_sz299_bs32_resnet152.pkl')
path_results = Path(f'./inference_results_{path_damglearn.stem}')
os.makedirs(path_results, exist_ok=True)

for idx in tqdm.tqdm(range(len(preimgs))):
    npre, npos = preimgs[idx], posimgs[idx]
    damgmask = infer_sample(npre, npos, seglearn, path_damglearn)
    imgid = get_imgid(npre)
    damgmask.save(path_results/f'test_localization_{imgid}_prediction.png')
    damgmask.save(path_results/f'test_damage_{imgid}_prediction.png')

  0%|          | 0/933 [00:00<?, ?it/s]

  0%|          | 2/933 [00:02<19:11,  1.24s/it]

  0%|          | 3/933 [00:03<18:57,  1.22s/it]

  0%|          | 4/933 [00:05<24:40,  1.59s/it]

  1%|          | 5/933 [00:07<24:20,  1.57s/it]

...

 99%|█████████▉| 927/933 [33:29<00:14,  2.46s/it]

 99%|█████████▉| 928/933 [33:30<00:10,  2.10s/it]

100%|█████████▉| 930/933 [33:32<00:04,  1.46s/it]

100%|█████████▉| 932/933 [33:35<00:01,  1.42s/it]

100%|██████████| 933/933 [33:36<00:00,  2.16s/it]


### Load a few saved submission png files

In [125]:
print(len(get_image_files(path_results)))
get_image_files(path_results)[:5]

1866


[PosixPath('inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00536_prediction.png'),
 PosixPath('inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00313_prediction.png'),
 PosixPath('inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00142_prediction.png'),
 PosixPath('inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00376_prediction.png'),
 PosixPath('inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00919_prediction.png')]

In [126]:
#imgid = get_imgid(preimgs[117])
imgid = get_imgid(preimgs[332]) #Lots of buildings [0, 1, 3, 4]
# imgid = '00064'  no buildings

In [127]:
path_local = path_results/f'test_localization_{imgid}_prediction.png'
path_damg = path_results/f'test_damage_{imgid}_prediction.png'

In [128]:
np.unique(PIL.Image.open(path_damg))

array([0, 1, 2, 3, 4], dtype=uint8)

In [129]:
CMAP.colors

['None', 'b', 'c', 'orange', 'r']

In [130]:
_, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 15))
axs[0].imshow(np.array(PIL.Image.open(path_local)), cmap=CMAP, vmin=0, vmax=4)
axs[1].imshow(np.array(PIL.Image.open(path_damg)), cmap=CMAP, vmin=0, vmax=4);

In [131]:
mask_local, mask_damg = open_mask(path_local), open_mask(path_damg)

In [132]:
mask_local.data.unique(), mask_damg.data.unique()

(tensor([0, 1, 2, 3, 4]), tensor([0, 1, 2, 3, 4]))

In [133]:
CMAP.colors

['None', 'b', 'c', 'orange', 'r']

In [134]:
_, axs = plt.subplots(ncols=2, figsize=(12, 6))
open_mask(path_local).show(axs[0], cmap=CMAP, vmax=len(CMAP.colors) - 1)
open_mask(path_damg).show(axs[1], cmap=CMAP, vmax=len(CMAP.colors) - 1)

### Save submission png files into tarball

In [136]:
#! tar czvf {str(path_results)}.tar.gz {str(path_results)}


inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00536_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00313_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00142_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00376_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00919_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00111_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00065_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00620_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00315_predict

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00237_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00883_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00383_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00652_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00895_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00139_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00565_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00457_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00123_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_res

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00255_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00054_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00142_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00669_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00738_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00620_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00621_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00802_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00653_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00044_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00461_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00753_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00301_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00563_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00544_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00163_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00799_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00240_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00539_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00428_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00437_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00455_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00207_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00239_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00660_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00930_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00520_prediction.png
inference_results_best_model_damg_nsample94

...

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00894_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00733_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00699_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00041_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00901_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00574_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00108_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00623_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00797_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00191_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00909_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00670_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00593_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00531_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00896_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00107_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00602_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00824_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00585_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00546_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00906_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00319_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00446_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00920_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00158_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00653_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00826_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00375_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00498_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00628_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00905_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00760_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00236_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00260_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00250_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00098_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs

inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00591_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00654_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00554_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00651_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00583_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00878_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00336_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_damage_00665_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_localization_00902_prediction.png
inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152/test_

In [137]:
! du -hs inference_results*

18M	inference_results
12M	inference_results.tar.gz
17M	inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152
11M	inference_results_best_model_damg_nsample94313_sz299_bs32_resnet152.tar.gz
17M	inference_results_dcedf650b7
11M	inference_results_dcedf650b7.tar.gz
17M	inference_results_stage2_best_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth
11M	inference_results_stage2_best_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth.tar.gz
17M	inference_results_stage2_last_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth
11M	inference_results_stage2_last_model_damg_nsample90000_sz128_bs32_resnet50_tfms_mixup_labsmooth.tar.gz


# - fin

In [138]:
! python notebook2script.py 03b_inference_pipeline.ipynb

Converted 03b_inference_pipeline.ipynb to xview2comp/nb_03b.py
